In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import time

In [2]:
api_key = "aeRc3h5tR5wweegUs1yyQ98QGs1H4ti2jpGW5hGn"
base_url = "https://api.nasa.gov/neo/rest/v1/feed"
start_date = "2025-03-17"
end_date = "2025-03-24"
iterations = 7

all_neo_data = []

url = f"{base_url}?start_date={start_date}&end_date={end_date}&api_key={api_key}"

for _ in range(iterations):
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        
        for date, neos in data["near_earth_objects"].items():
            for neo in neos:
                all_neo_data.append({
                    "id": neo["id"],
                    "name": neo["name"],
                    "absolute_magnitude_h": neo["absolute_magnitude_h"],
                    "estimated_diameter_km_min": neo["estimated_diameter"]["kilometers"]["estimated_diameter_min"],
                    "estimated_diameter_km_max": neo["estimated_diameter"]["kilometers"]["estimated_diameter_max"],
                    "is_potentially_hazardous": neo["is_potentially_hazardous_asteroid"],
                    "close_approach_date": neo["close_approach_data"][0]["close_approach_date"] if neo["close_approach_data"] else None,
                    "relative_velocity_kph": neo["close_approach_data"][0]["relative_velocity"]["kilometers_per_hour"] if neo["close_approach_data"] else None,
                    "miss_distance_km": neo["close_approach_data"][0]["miss_distance"]["kilometers"] if neo["close_approach_data"] else None,
                    "orbiting_body": neo["close_approach_data"][0]["orbiting_body"] if neo["close_approach_data"] else None
                })

        url = data["links"].get("next")
        time.sleep(0.5)
        if not url:
            break
    else:
        print("Error fetching data:", response.status_code, response.text)
        break

df = pd.DataFrame(all_neo_data)
df.head()
# df.to_csv("neo_data.csv", index=False)

,id,name,absolute_magnitude_h,estimated_diameter_km_min,estimated_diameter_km_max,is_potentially_hazardous,close_approach_date,relative_velocity_kph,miss_distance_km,orbiting_body
0,2462238,462238 (2008 CN1),20.66,0.196135,0.438571,True,2025-03-20,66079.8670338079,49188925.87487994,Earth
1,3840738,(2019 FN2),23.70,0.048368,0.108153,False,2025-03-20,51886.6892112298,55448221.33156316,Earth
2,54016204,(2020 FM),24.10,0.040230,0.089958,False,2025-03-20,40368.0578565874,44904232.423796424,Earth
3,54132069,(2021 FM),24.76,0.029686,0.066380,False,2025-03-20,49329.707888061,12660924.116322009,Earth
4,54157829,(2021 LR3),24.35,0.035855,0.080175,False,2025-03-20,21662.5248496921,66177846.041735665,Earth


In [3]:
df.shape[0]
with open("neo_data.js", "w") as js_file:
    js_file.write("const neoData = ")
    json.dump(all_neo_data, js_file, indent=2)
    js_file.write(";\nexport default neoData;")